In [13]:
import requests
from bs4 import BeautifulSoup
import os
import  xlsxwriter
import pandas as pd

base_URL = "https://www.goerg.de"


URL = "https://www.goerg.de/en/our-people"

page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
content = soup.find_all('span',class_="field-content")
url = []

nam=[]
posi=[]
ofst=[]
zipp=[]
cit=[]
ph=[]

em=[]

all_links=[]

user_url = []
while(1):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    content = soup.find_all('span',class_="field-content")
    load_page = soup.find('a',rel='next')
    for i in content:
        user_url.append(base_URL + i.find_next('a')['href'])
    URL = URL+load_page['href']
        
    if("4" in URL):
        break

#loopsub

for i in user_url:
    page = requests.get(i)
    soup = BeautifulSoup(page.content, "html.parser")
    nn = soup.find(class_="page-person-title").text
    nam.append(nn)
    
    pos = soup.find(class_="field-subline").text
    sp=pos.split("|")
    sp2=sp[1]
    posi.append(sp2)

    street=soup.find(class_="field-street").text
    ofst.append(street)
    
    zz= soup.find(class_="field-zip").text
    zipp.append(zz)
    
    city=soup.find(class_="field-city").text
    cit.append(city)
    
    phn=soup.find(class_="field-phone").find('a').text
    ph.append(phn)
    
   
    email=soup.find(class_="field-email").find(class_="spamspan").text
    em.append(email)
for lin in user_url:
    all_links.append(lin)  

In [14]:
import pandas as pd
df = pd.DataFrame()
df
df['Sno'] = df.index + 1
df.set_index('Sno',inplace=True)
df['NAME']=nam
df['position']=posi
df['streetname']=ofst
df['City']=cit
df['zipcode']=zipp
df['phoneNo']=ph
df['Email']=em
df['links']=all_links
df

NAME  \
Sno                                       
0    Dr. Marie Ackermann, LL.M. (Paris)   
1                        Hartwig Albers   
2                        Thomas Altmann   
3                    Dr. Markus Andrees   
4              Christian Assmann, M. A.   
..                                  ...   
67                    Dr. Dirk Freihube   
68                       Maren Freymuth   
69                     Dr. Stefan Frick   
70                      Barbara Frieser   
71                         Daniel Fuchs   

                                        position                 streetname  \
Sno                                                                           
0                           Associated Partner         Alter Wall 20 – 22     
1                                      Partner             Kantstrasse 164    
2                                   Associate      Prinzregentenstrasse 22    
3                                    Associate              Kennedyplatz 2    
4                        Employed Professional         Alter Wall 20 – 22     
..                                           ...                        ...   
67                    Specialist for labour law            Ulmenstrasse 30    
68                                   Associate              Kennedyplatz 2    
69                         Associated Partner          Alter Wall 20 – 22     
70                                   Associate     Prinzregentenstrasse 22    
71    Specialist for information technology law            Kantstrasse 164    

                    City   zipcode              phoneNo  \
Sno                                                       
0               Hamburg     20457     +49 40 500360 480   
1                Berlin    10623       +49 30 311000 12   
2                Munich    80538       +49 89 3090667 0   
3               Cologne     50679     +49 221 33660 244   
4               Hamburg     20457     +49 40 500360 325   
..                   ...       ...                  ...   
67    Frankfurt am Main     60325     +49 69 170000 159   
68              Cologne     50679     +49 221 33660 284   
69              Hamburg     20457     +49 40 500360 725   
70               Munich    80538    +49 89 30 90 667 25   
71               Berlin    10623      +49 30 884503 143   

                           Email  \
Sno                                
0       mackermann [at] goerg.de   
1          halbers [at] goerg.de   
2         taltmann [at] goerg.de   
3         mandrees [at] goerg.de   
4    cassmann [at] bwls-goerg.de   
..                           ...   
67       dfreihube [at] goerg.de   
68       mfreymuth [at] goerg.de   
69          sfrick [at] goerg.de   
70        bfrieser [at] goerg.de   
71          dfuchs [at] goerg.de   

                                                 links  
Sno                                                     
0    https://www.goerg.de/en/our-people/dr-marie-ac...  
1    https://www.goerg.de/en/our-people/hartwig-albers  
2    https://www.goerg.de/en/our-people/thomas-altmann  
3    https://www.goerg.de/en/our-people/dr-markus-a...  
4    https://www.goerg.de/en/our-people/christian-a...  
..                                                 ...  
67   https://www.goerg.de/en/our-people/dr-dirk-fre...  
68   https://www.goerg.de/en/our-people/maren-freymuth  
69   https://www.goerg.de/en/our-people/dr-stefan-f...  
70   https://www.goerg.de/en/our-people/barbara-fri...  
71     https://www.goerg.de/en/our-people/daniel-fuchs  

[72 rows x 8 columns]

In [15]:
import os
#import sqlite3
import pandas as pd
import sqlite3 as sql
from flask import jsonify
from flask_api import FlaskAPI
from flask_restful import Api
from sqlalchemy import create_engine
import json
app=FlaskAPI(__name__)
api=Api(app)
c=create_engine('sqlite:///Technical.db')
@app.route('/home')
def Home():
    return '<br><h2><center>DATA SCRAPING USING PYTHON</center></h2></br>'
@app.route('/select', methods=['GET'])
def db_select_one():
    conn = c.connect()
    y = conn.execute("select * from Learn_data")  
    return json.dumps([dict(r) for r in y])

@app.route('/push', methods=['GET'])
def tbl_insert():
    conn = c.connect()
    df.to_sql('Learn_data', conn,if_exists='replace')
    text = {"Message " : "Record inserted successfully"}
    return text

@app.route('/create', methods=['GET'])
def create():
    try:
        conn=c.connect()
        trans=conn.begin()
        conn.execute('CREATE TABLE IF NOT EXISTS Learn_data(Sno INTEGER PRIMARY KEY AUTOINCREMENT,Name text, position text,streetname text,city text,zipcode number,phoneNo number,Email text,URL text)')
        trans.commit()
        return("User table created successfully")
    except:
         return("User table creation failed")
    finally:
        conn.close()
app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
